In [132]:
import numpy as np
from numpy import linalg as LA
from imp import reload
from numpy import array, identity, ones, nonzero, zeros
from numpy import linalg as LA
from collections import defaultdict
import random

from PageRank_module import pageRank

file = "top250movies.txt"

In [133]:
try:
    with open(file, encoding = "utf-8") as f:
        movies = defaultdict(list)
        for line in f:
            line = line.strip().split('/')
            movie = line[0]
            cast = line[1:]
            movies[movie].extend(cast)
except FileNotFoundError:
    raise IOError(f"Error: Could not find file {file}.")
except ValueError:
    raise ValueError(f"Error: Invalid data in file {file}.")


In [134]:
print("Number of movies: ", len(movies))
print("Number of actors: ", len(set([a for m in movies for a in movies[m]])))
print("Example entry: ", list(movies.items())[random.randint(0, len(movies))])

Number of movies:  250
Number of actors:  14882
Example entry:  ('The Nights of Cabiria (1957)', ['Giulietta Masina', 'François Périer', 'Franca Marzi', 'Dorian Gray', 'Aldo Silvani', 'Ennio Girolami', 'Mario Passante', 'Christian Tassou', 'Amedeo Nazzari', 'Gianni Baghino', 'Franco Balducci', 'Ciccio Barbi', 'Jusy Boncinelli', 'Loretta Capitoli', 'Leo Catozzo', 'Dominique Delouche', 'Edda Evangelista', 'Franco Fabrizi', 'Riccardo Fellini', 'Ines Ferrari', 'Biagio Gambini', 'Giovanna Gattinoni', 'Amedeo Girardi', 'Pina Gualandri', 'Elio Mauro', 'Nino Milano', 'Jean Mollier', 'Sandro Moretti', 'Sergio Parlato', 'Mimmo Poli', 'Polidor', 'María Luisa Rolando', 'Vittorio Tosti'])


In [135]:
# Create adjacency matrix
actors = sorted(set([actor for cast in movies.values() for actor in cast]))
n = len(actors)
G = zeros((n, n))

for cast in movies.values():
    for i, actor1 in enumerate(cast):
        if actor1 in actors:
            for actor2 in cast[i + 1:]:
                if actor2 in actors and actor1 != actor2:
                    G[actors.index(actor1), actors.index(actor2)] += 1

In [146]:
# Compute PageRank
netw = pageRank(G, actors,0.85,1e-6,15)
prv = netw.linsolve()

In [147]:
# Write the ranked movies to a file
rankedActors = sorted(zip(actors, prv), key = lambda x: x[1], reverse = True)
print("Top 10 actors:")
for actor, rank in rankedActors[:10]:
    print(f"{actor}: {rank}")
    
print("Bottom 10 actors:")
for actor, rank in rankedActors[-10:]:
    print(f"{actor}: {rank}")
    
with open("rankedMovies.txt", "w") as f:
    for actor, rank in rankedActors:
        f.write(f"{actor}\n")

Top 10 actors:
Jef Groff: ('Ümit Çirak', 3.0203981396791377e-05)
Danny Flaherty: ('Ülkü Duru', 6.622230005984798e-05)
Kemal Inci: ('Øyvind Hagen-Traberg', 2.4684146515192746e-05)
Diane Poulos: ('Özkan Ugur', 5.8004934358990924e-05)
Robert Shayne: ('Özgül Arslan', 1.6658550763269785e-05)
Jim Thornton: ('Özge Özberk', 2.9032782446397224e-05)
Samui Muang-Intata: ('Íñigo Garcés', 1.5939942267674478e-05)
John Houchin: ('Émilie Caen', 2.757783569185633e-05)
Leslie Ishii: ('Éder Júlio Martins', 2.298070730118878e-05)
Andrew Guzman: ('Çetin Tekindor', 0.0002524618046469687)
Bottom 10 actors:
Laura Linney: ('Aakash Dabhade', 2.3526200410298337e-05)
Michael McShane: ('Aadil', 2.006202947545214e-05)
Gene Kelly: ('A.S. Duggal', 4.160728074201984e-05)
Lee Asquith-Coe: ('A.R. Haysel', 2.3295393765489663e-05)
Paul McMichael: ("A.J. O'Connor", 1.8218814257187047e-05)
Nelly Arno: ('A.B. Lane', 1.9141681029992788e-05)
Yeon-Seok Yoo: ('A. Marshal Ward', 1.3671646342402958e-05)
John Tomlinson: ('A. Camero